## Từ đây xuống bỏ

# Bài hoàn chỉnh

In [ ]:
# =========================================================
# Analysts@Emory — BTC next-hour up/down (±0.01%) classifier (improved)
# =========================================================

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import f1_score, classification_report
from sklearn.impute import SimpleImputer

import lightgbm as lgb
from lightgbm import early_stopping

# -------------------------
# 1) ĐỌC DỮ LIỆU
# -------------------------
CANDIDATE_PATHS = [
    "/content/data/input/emory/BTC_USDT_1h.csv"
]

data_path = None
for p in CANDIDATE_PATHS:
    if os.path.exists(p):
        data_path = p
        break
if data_path is None:
    raise FileNotFoundError("Không tìm thấy BTC_USDT_1h.csv trong Input.")

df = pd.read_csv(data_path)

# -------------------------
# 2) PARSE TIME
# -------------------------
if "unix" in df.columns:
    mx = pd.to_numeric(df["unix"], errors="coerce").max()
    unit = "ms" if mx > 1e12 else "s"
    df["date"] = pd.to_datetime(df["unix"], unit=unit, errors="coerce")
else:
    df["date"] = pd.to_datetime(df["date"], format="mixed", errors="coerce")

df = df.dropna(subset=["date"]).sort_values("date").set_index("date")

rename_map = {}
for c in df.columns:
    if c.strip().lower() == "volume btc":
        rename_map[c] = "volume_btc"
    if c.strip().lower() == "volume usdt":
        rename_map[c] = "volume_usdt"
df = df.rename(columns=rename_map)

# -------------------------
# 3) TARGET
# -------------------------
df["ret_fwd1"] = df["close"].pct_change(periods=1).shift(-1)
thr = 1e-4
df["target"] = np.where(df["ret_fwd1"] >= thr, 1,
                        np.where(df["ret_fwd1"] <= -thr, 0, np.nan))
df = df.dropna(subset=["target"])

print("Target distribution:")
print(df["target"].value_counts().rename_axis("target").to_frame("count"))
print((df["target"].value_counts(normalize=True)).rename("proportion"))

# -------------------------
# 4) FEATURE ENGINEERING
# -------------------------
def add_features(frame, prefix="btc"):
    c = "close"; h="high"; l="low"; o="open"
    v1 = "volume_btc"; v2 = "volume_usdt"
    out = pd.DataFrame(index=frame.index)

    if c in frame.columns:
        logp = np.log(frame[c].clip(lower=1e-9))
        out[f"{prefix}_ret_1"]   = logp.diff(1)
        out[f"{prefix}_ret_3"]   = logp.diff(3)
        out[f"{prefix}_ret_6"]   = logp.diff(6)
        out[f"{prefix}_ret_12"]  = logp.diff(12)
        out[f"{prefix}_ma_6"]    = frame[c].rolling(6).mean()
        out[f"{prefix}_ma_12"]   = frame[c].rolling(12).mean()
        out[f"{prefix}_std_6"]   = frame[c].rolling(6).std()
        out[f"{prefix}_std_12"]  = frame[c].rolling(12).std()

        # RSI(14)
        delta = frame[c].diff()
        up = delta.clip(lower=0); down = -delta.clip(upper=0)
        rs = up.rolling(14).mean() / (down.rolling(14).mean() + 1e-9)
        out[f"{prefix}_rsi_14"] = 100 - (100 / (1 + rs))

        # MACD
        ema12 = frame[c].ewm(span=12, adjust=False).mean()
        ema26 = frame[c].ewm(span=26, adjust=False).mean()
        macd = ema12 - ema26; signal = macd.ewm(span=9, adjust=False).mean()
        out[f"{prefix}_macd"] = macd
        out[f"{prefix}_macd_sig"] = signal
        out[f"{prefix}_macd_hist"] = macd - signal

        # Candle shape features
        if all(k in frame.columns for k in [o,h,l,c]):
            out[f"{prefix}_hl"] = frame[h] - frame[l]
            out[f"{prefix}_co"] = frame[c] - frame[o]
            out[f"{prefix}_upper_shadow"] = frame[h] - frame[[c,o]].max(axis=1)
            out[f"{prefix}_lower_shadow"] = frame[[c,o]].min(axis=1) - frame[l]

        # Volatility cluster
        ret2 = out[f"{prefix}_ret_1"]**2
        out[f"{prefix}_vol_cluster_12"] = ret2.rolling(12).mean()
        out[f"{prefix}_vol_cluster_24"] = ret2.rolling(24).mean()

    # volumes
    if v1 in frame.columns:
        out[f"{prefix}_vol_btc"] = frame[v1]
        out[f"{prefix}_vol_btc_chg"] = frame[v1].pct_change(1)
    if v2 in frame.columns:
        out[f"{prefix}_vol_usdt"] = frame[v2]
        out[f"{prefix}_vol_usdt_chg"] = frame[v2].pct_change(1)

    # time features
    out["hour"] = out.index.hour
    out["dow"]  = out.index.dayofweek
    return out

X_all = add_features(df, "btc")
XY = pd.concat([X_all, df["target"]], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
y = XY["target"].astype(int)
X = XY.drop(columns=["target"])

imp = SimpleImputer(strategy="median")
X_imp = pd.DataFrame(imp.fit_transform(X), index=X.index, columns=X.columns)
print("Data ready:", X_imp.shape, "| positives:", int(y.sum()), "| negatives:", int((1-y).sum()))

# -------------------------
# 5) CV + LightGBM
# -------------------------
tscv = TimeSeriesSplit(n_splits=5)
oof_pred_bin = pd.Series(index=y.index, dtype=float)
oof_pred_proba = pd.Series(index=y.index, dtype=float)
f1_scores = []

for fold, (tr_idx, va_idx) in enumerate(tscv.split(X_imp), 1):
    X_tr, X_va = X_imp.iloc[tr_idx], X_imp.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    model = lgb.LGBMClassifier(
        n_estimators=3000,
        learning_rate=0.02,
        num_leaves=63,
        max_depth=-1,
        min_child_samples=50,
        subsample=0.7,
        subsample_freq=1,
        colsample_bytree=0.7,
        reg_alpha=0.5,
        reg_lambda=1.0,
        objective="binary",
        random_state=42
    )

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="binary_logloss",
        callbacks=[early_stopping(100, verbose=False)]
    )

    val_proba = model.predict_proba(X_va)[:, 1]
    thr_grid = np.linspace(0.05, 0.95, 91)  # chi tiết hơn
    best_f1, best_thr = -1, 0.5
    for t in thr_grid:
        f1 = f1_score(y_va, (val_proba >= t).astype(int))
        if f1 > best_f1:
            best_f1, best_thr = f1, t

    oof_pred_proba.iloc[va_idx] = val_proba
    oof_pred_bin.iloc[va_idx]   = (val_proba >= best_thr).astype(int)
    f1_scores.append(best_f1)
    print(f"Fold {fold}: best F1={best_f1:.4f} @thr={best_thr:.2f}")

# -------------------------
# Evaluation
# -------------------------
valid_idx = oof_pred_bin.dropna().index
print("\nMean F1 (OOF):", np.mean(f1_scores).round(4))
print(classification_report(y.loc[valid_idx],
                            oof_pred_bin.loc[valid_idx].astype(int)))


Target distribution:
        count
target       
1.0     19765
0.0     18732
target
1.0    0.513417
0.0    0.486583
Name: proportion, dtype: float64
Data ready: (38451, 24) | positives: 19734 | negatives: 18717
[LightGBM] [Info] Number of positive: 3338, number of negative: 3073
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001583 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5619
[LightGBM] [Info] Number of data points in the train set: 6411, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.520668 -> initscore=0.082718
[LightGBM] [Info] Start training from score 0.082718
Fold 1: best F1=0.6729 @thr=0.41
[LightGBM] [Info] Number of positive: 6574, number of negative: 6245
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003035 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5625
[L

In [ ]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

# ===== Config =====
predict_path = "/content/data/input/emory/predictions.csv"   # TODO: đổi path file predict gốc (vd: "/mnt/data/annt43_predict.csv")
out_dir = Path("/content/data/output")
out_dir.mkdir(parents=True, exist_ok=True)
TARGET_ROWS = 39392
THRESH = 0.5  # ngưỡng nhị phân cho dự đoán từ model
FMT = '%m/%d/%Y %H:%M'
REGEX = r'^(0[1-9]|1[0-2])/(0[1-9]|[12]\d|3[01])/\d{4} (?:[01]\d|2[0-3]):[0-5]\d$'

def normalize_date_text(values):
    """
    Nhận Series hoặc Index, chuẩn hoá chuỗi thời gian lạ (ví dụ '2017-08-31 03-PM', '03PM', ...),
    parse -> datetime, làm tròn đến phút. Trả về cùng "kiểu": nếu input là Index -> DatetimeIndex,
    nếu input là Series -> Series[datetime64[ns]].
    """
    # Luôn làm việc trên Series dạng string để replace dễ
    s = pd.Series(values, copy=False).astype(str).str.strip()

    # Chuẩn hoá các biến thể AM/PM
    s = s.str.replace(r'(\d{1,2})-(AM|PM)\b', r'\1:00 \2', regex=True, flags=re.IGNORECASE)
    s = s.str.replace(r'\b(\d{1,2})\s*(AM|PM)\b', r'\1:00 \2', regex=True, flags=re.IGNORECASE)
    s = s.str.replace(r'\b(\d{1,2}):(\d{2})(AM|PM)\b', r'\1:\2 \3', regex=True, flags=re.IGNORECASE)

    # Parse ưu tiên MM/DD, sau đó thử dayfirst để cứu trường hợp hiếm
    dt = pd.to_datetime(s, errors='coerce', dayfirst=False, infer_datetime_format=True)
    miss = dt.isna()
    if miss.any():
        dt2 = pd.to_datetime(s[miss], errors='coerce', dayfirst=True, infer_datetime_format=True)
        dt[miss] = dt2

    # Làm tròn đến phút
    dt = dt.dt.floor('min')

    # Trả về cùng "kiểu" với input
    if isinstance(values, pd.Index):
        return pd.DatetimeIndex(dt)
    else:
        return dt


def parse_0_1_or_nan(series: pd.Series) -> pd.Series:
    """Trả về 0/1 nếu giá trị đúng 0 hoặc 1; ngược lại -> NaN (coi như 'chưa predict được')."""
    vals = pd.to_numeric(series, errors='coerce')
    out = pd.Series(np.nan, index=series.index, dtype='float64')
    out[(vals == 0) | (vals == 1)] = vals[(vals == 0) | (vals == 1)]
    return out

def model_predict_binary(X) -> np.ndarray:
    """Trả về vector 0/1 từ model (support cả classifier có predict_proba và regressor)."""
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)[:, 1]
        return (proba >= THRESH).astype(int)
    raw = np.asarray(model.predict(X)).ravel()
    return raw.astype(int) if set(np.unique(raw)).issubset({0,1}) else (raw >= THRESH).astype(int)

# ===== 1) Đọc file predict gốc =====
pred_src = pd.read_csv(predict_path)

# Xác định cột date (ưu tiên tên 'date', nếu không có thì lấy cột đầu tiên)
date_col = 'date' if 'date' in pred_src.columns else pred_src.columns[0]
dates_parsed = normalize_date_text(pred_src[date_col])

# Xác định cột dự đoán của file gốc (ưu tiên '0.02%', sau đó 'target', nếu không thì cột thứ 2)
if '0.02%' in pred_src.columns:
    src_pred_col = '0.02%'
elif 'target' in pred_src.columns:
    src_pred_col = 'target'
elif len(pred_src.columns) >= 2:
    src_pred_col = pred_src.columns[1]
else:
    raise ValueError("Không tìm thấy cột dự đoán trong file predict (cần >= 2 cột).")

# Giá trị cũ hợp lệ (0/1) -> giữ; còn lại (NaN/khác 0-1) -> coi là thiếu để điền bằng model
src_01 = parse_0_1_or_nan(pred_src[src_pred_col])

# ===== 2) Chuẩn hoá index của X_imp để map theo phút =====
X_imp_norm = X_imp.copy()
X_imp_norm.index = normalize_date_text(pd.Index(X_imp_norm.index))

# Tập mốc thời gian có trong file predict
need_dates = pd.to_datetime(dates_parsed.dropna().unique())
X_test = X_imp_norm.loc[X_imp_norm.index.isin(need_dates)].copy()

# ===== 3) Dự đoán từ model & tạo map theo date =====
if len(X_test) > 0:
    model_bin = model_predict_binary(X_test)
    pred_map = pd.Series(model_bin, index=pd.to_datetime(X_test.index))
    # nếu có duplicate index, lấy cái cuối
    pred_map = pred_map[~pred_map.index.duplicated(keep='last')]
else:
    pred_map = pd.Series(dtype='float64')

# ===== 4) Hợp nhất: giữ giá trị cũ 0/1; chỗ nào NaN thì điền bằng model; còn thiếu -> 0
final_vals = src_01.copy()
mask_missing = final_vals.isna()

if mask_missing.any():
    # lấy dự đoán theo date tương ứng
    fill_dates = pd.to_datetime(dates_parsed[mask_missing], errors='coerce')
    fill_from_model = []
    for ts in fill_dates:
        if pd.isna(ts) or (ts not in pred_map.index):
            fill_from_model.append(np.nan)
        else:
            fill_from_model.append(int(pred_map.loc[ts]))
    final_vals.loc[mask_missing] = fill_from_model

# chỗ nào vẫn NaN (không map được model) -> 0
final_vals = final_vals.fillna(0).astype(int)

# ===== 5) Lập DataFrame output với format date chuẩn & đúng header =====
out = pd.DataFrame({
    'date': pd.to_datetime(dates_parsed, errors='coerce').dt.strftime(FMT),
    '0.02%': final_vals.values
})

# ===== 6) Kiểm tra format date toàn bộ & đảm bảo đúng 39,392 dòng =====
ok_regex = out['date'].str.match(REGEX, na=False)
ok_parse = pd.to_datetime(out['date'], format=FMT, errors='coerce').notna()
bad_mask = ~(ok_regex & ok_parse)
if bad_mask.any():
    print("❌ Ví dụ date sai format:")
    print(out.loc[bad_mask, 'date'].head(10).to_string(index=False))
    raise ValueError("Date format invalid. Yêu cầu 'MM/DD/YYYY HH:MM' (vd: 10/13/2021 03:00).")

if len(out) > TARGET_ROWS:
    out = out.iloc[:TARGET_ROWS].copy()
elif len(out) < TARGET_ROWS:
    raise ValueError(f"Row count {len(out)} < {TARGET_ROWS}. File predict gốc phải đủ {TARGET_ROWS} dòng.")

# ===== 7) Lưu =====
save_path = out_dir / "submission.csv"
out.to_csv(save_path, index=False)
print(f"✅ Saved {save_path} | rows={len(out)} | header=['date','0.02%'] | format={FMT} | kept old 0/1, filled missing via model, else 0")


/tmp/ipython-input-2320659374.py:30: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(s, errors='coerce', dayfirst=False, infer_datetime_format=True)
/tmp/ipython-input-2320659374.py:33: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt2 = pd.to_datetime(s[miss], errors='coerce', dayfirst=True, infer_datetime_format=True)
/tmp/ipython-input-2320659374.py:33: UserWarning: Parsing dates in %Y-%m-%d %I:%M:%S %p format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt2 = pd.to_datetime(s[m

✅ Saved /content/data/output/submission.csv | rows=39392 | header=['date','0.02%'] | format=%m/%d/%Y %H:%M | kept old 0/1, filled missing via model, else 0


In [ ]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

# ===== CONFIG =====
template_path = "/content/data/input/emory/predictions.csv"  # file nguồn có cột 'date'
out_dir = Path("/content/data/output"); out_dir.mkdir(parents=True, exist_ok=True)
save_path = out_dir / "submission.csv"
THRESH = 0.5
EXPECTED_ROWS = 39392

def to_datetime_flex(series_like):
    s = pd.Series(series_like, copy=False).astype(str).str.strip()
    s = s.str.replace(r'(\d{1,2})-(AM|PM)\b', r'\1:00 \2', regex=True, flags=re.IGNORECASE)
    s = s.str.replace(r'\b(\d{1,2})\s*(AM|PM)\b', r'\1:00 \2', regex=True, flags=re.IGNORECASE)
    s = s.str.replace(r'\b(\d{1,2}):(\d{2})(AM|PM)\b', r'\1:\2 \3', regex=True, flags=re.IGNORECASE)
    dt = pd.to_datetime(s, errors='coerce', dayfirst=False, infer_datetime_format=True)
    miss = dt.isna()
    if miss.any():
        dt2 = pd.to_datetime(s[miss], errors='coerce', dayfirst=True, infer_datetime_format=True)
        dt[miss] = dt2
    return dt.dt.floor('min')

def to_binary_or_zero(series):
    """0/1 giữ nguyên; nếu là số thực -> threshold; còn lại/NaN -> 0."""
    arr = pd.to_numeric(series, errors='coerce')
    if arr.notna().all():
        uniq = set(np.unique(arr.values))
        if uniq.issubset({0,1}):
            return arr.fillna(0).astype(int)
        return (arr >= THRESH).astype(int)
    return pd.Series(np.zeros(len(series), dtype=int), index=series.index)

# ===== 1) Đọc template & xác nhận kích thước =====
tpl = pd.read_csv(template_path)
date_raw = tpl.iloc[:, 0].astype(str)     # giữ nguyên chuỗi 'date'
n = len(tpl)
if n != EXPECTED_ROWS:
    print(f"⚠️ File nguồn có {n} dòng (mong đợi {EXPECTED_ROWS}). Vẫn xử lý theo {n} dòng.")

# Nếu file nguồn đã có cột dự đoán, dùng để "giữ nguyên" nửa trên
if '0.02%' in tpl.columns:
    src_pred = to_binary_or_zero(tpl['0.02%'])
elif 'target' in tpl.columns:
    src_pred = to_binary_or_zero(tpl['target'])
elif tpl.shape[1] >= 2:
    src_pred = to_binary_or_zero(tpl.iloc[:, 1])
else:
    src_pred = pd.Series(np.zeros(n, dtype=int))  # không có cột dự đoán -> nửa trên sẽ là 0

# ===== 2) Chuẩn bị map dự đoán từ model (chỉ dùng cho nửa dưới) =====
# Parse datetime chỉ để map, không đổi output 'date'
date_dt = to_datetime_flex(date_raw)

X_norm = X_imp.copy()
X_norm.index = to_datetime_flex(pd.Series(X_norm.index))

need_dt = pd.to_datetime(date_dt.dropna().unique())
X_test = X_norm.loc[X_norm.index.isin(need_dt)].copy()

if hasattr(model, "predict_proba"):
    proba = model.predict_proba(X_test)[:, 1]
    model_bin = (proba >= THRESH).astype(int)
else:
    raw = np.asarray(model.predict(X_test)).ravel()
    model_bin = raw.astype(int) if set(np.unique(raw)).issubset({0,1}) else (raw >= THRESH).astype(int)

pred_map = pd.Series(model_bin, index=pd.to_datetime(X_test.index))
pred_map = pred_map[~pred_map.index.duplicated(keep='last')]

# ===== 3) Chỉ predict các dòng có số thứ tự > 50% =====
half = n // 2  # 0..half-1 giữ nguyên, half..n-1 predict
out_vals = src_pred.copy().astype(int).values  # mặc định giữ nguyên

for i in range(half, n):
    ts = date_dt.iloc[i]
    out_vals[i] = int(pred_map.get(ts, 0))  # không map được -> 0

# ===== 4) Kết quả & lưu =====
out = pd.DataFrame({
    'date': date_raw,         # giữ nguyên chuỗi date như file nguồn
    '0.02%': out_vals.astype(int)
})

# đảm bảo số dòng đúng bằng file nguồn (thường là 39,392)
if len(out) > EXPECTED_ROWS:
    out = out.iloc[:EXPECTED_ROWS].copy()
elif len(out) < EXPECTED_ROWS:
    print(f"⚠️ Output có {len(out)} dòng < {EXPECTED_ROWS}. (Giữ nguyên bằng số dòng nguồn).")

out.to_csv(save_path, index=False)
print(f"✅ Saved {save_path} | rows={len(out)} | chỉ predict nửa dưới ({n-half} dòng); nửa trên giữ nguyên; header=['date','0.02%']")


/tmp/ipython-input-3085548020.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(s, errors='coerce', dayfirst=False, infer_datetime_format=True)
/tmp/ipython-input-3085548020.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt2 = pd.to_datetime(s[miss], errors='coerce', dayfirst=True, infer_datetime_format=True)
/tmp/ipython-input-3085548020.py:21: UserWarning: Parsing dates in %Y-%m-%d %I:%M:%S %p format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt2 = pd.to_datetime(s[m

✅ Saved /content/data/output/submission.csv | rows=39392 | chỉ predict nửa dưới (19696 dòng); nửa trên giữ nguyên; header=['date','0.02%']
